<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN07/7.1-Vortrainierte_CNN_pl.ipynb)

## Datenvorbereitung mit einem LightningDataModule

Wir setzen voraus, dass der Datensatz in einem Ordner vorliegt, der die Unterordner `train`, `validation` und `test` enthält.
Die Trainingsbilder werden mithilfe von Data Augmentation (Rotation, zufälliger Crop, horizontales Spiegeln) vorbereitet, während für Validierung und Test nur eine Reskalierung und Normalisierung erfolgt.


# 7. Verwendung eines vortrainierten CNNs

This notebook contains the code sample found in Chapter 5, Section 3 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). 

- Vortrainierte NNs sind ein gängiger und äusserst effektiver Ansatz, Deep Learning mit kleinen Bilddatenmengen zu betreiben.
- Ein vortrainiertes NN ist ein gespeichertes CNN, das vorher mit einer grossen Datenmenge trainiert wurde, typischerweise für eine umfangreiche Bildklassifizierungsaufgabe. Wenn diese ursprüngliche Datenmenge gross und allgemein genug ist, kann die durch das vortrainierte NN erlernte räumliche Merkmalshierarchie als allgemeines Modell der visuellen Welt dienen.
- Deshalb können sich die Merkmale für viele verschiedene Aufgaben des maschinellen Sehens als nützlich erweisen, obwohl für diese Aufgaben völlig andere Klassen von Bedeutung sind als für die ursprüngliche Aufgabe. 
- Sie könnten beispielsweise ein NN mit den ImageNet-Daten trainieren (deren Klassen grösstenteils Tiere und Alltagsgegenstände sind) und dieses vortrainierte NN auch für etwas ganz anderes wie z.B. die Erkennung von Möbelstücken wiederverwenden. 
- Diese **Übertragbarkeit der erlernten Merkmale auf andere Aufgaben** ist ein entscheidender Vorteil des Deep Learnings gegenüber vielen älteren Shallow-Learning-Ansätzen und sorgt dafür, dass Deep Learning sehr gut für Aufgaben mit kleinen Datenmengen geeignet ist.





### 7.1 VGG-Architektur

Hier betrachten wir ein grosses, mit der ImageNet-Datensammlung (1.4 Millionen gekennzeichnete Bilder und 1'000 verschiedene Klassen) trainiertes CNN. Die Datenmenge enthält viele Tierklassen inklusive unterschiedlicher Hunde- und Katzenrassen, daher ist zu erwarten, dass sich das CNN gut für die Klassifizierung von Hunde- und Katzenbildern eignet. 
                                                                
Wir werden die 2014 von *Karen Simonyan* und *Andrew Zisserman* entwickelte [VGG16-Architektur](https://arxiv.org/abs/1409.1556) verwenden. Dabei handelt es sich um eine einfache und weithin gebräuchliche CNN-Architektur für die ImageNet-Datensammlung [1]. Das Modell ist zwar schon älter, weit vom heutigen Stand der Technik entfernt und
zudem etwas schwerfälliger als viele jüngere Modelle, dennoch habe ich es ausgewählt, weil die Architektur dem sehr ähnlich ist, was Sie bereits kennen. Sie ist gut verständlich, ohne dass es erforderlich wäre, neue Konzepte einzuführen. 

<img src="Bilder/VGG16.png" width="840" align="center"/>

Vielleicht hören Sie zum ersten Mal von einem dieser Modelle mit den eigentümlichen
Namen *VGG, ResNet, Inception, Inception-ResNet, Xception* [2] und wie sie alle heissen. Sie werden sich daran gewöhnen, denn sie werden Ihnen häufig begegnen, wenn Sie sich mit Deep Learning und maschinellem Sehen befassen.

[1] Karen Simonyan und Andrew Zisserman, *Very Deep Convolutional Networks for Large-Scale Image Recognition*, arXiv (2014), https://arxiv.org/abs/1409.1556.

[2] CNN Architectures: VGG, Resnet, InceptionNet, XceptionNet UseCases : Image Feature Extraction + Transfer Learning https://www.kaggle.com/shivamb/cnn-architectures-vgg-resnet-inception-tl


### Merkmalsextraktion und Feinabstimmung

Es gibt zwei Möglichkeiten, ein vortrainiertes NN zu verwenden: 
- **Merkmalsextraktion (feature extraction)** und **Feinabstimmung (fine tuning)**. 
- Wir werden beide betrachten und werfen zunächst einmal einen Blick auf die Merkmalsextraktion.

## 7.2 Feature extraction (Merkmalsextraktion)

Bei der Merkmalsextraktion werden die von einem vorangegangenen NN erlernten Repräsentationen dazu verwendet, neuen Samples interessante Merkmale zu entnehmen. Diese Merkmale werden anschliessend in einen von Grund auf neu trainierten Klassifizierer eingespeist.

Wie Sie bereits wissen, besitzen CNNs zur Bildklassifizierung zwei Bestandteile: 
- Am Anfang steht eine Reihe von Pooling- und Convolutional Layern, und sie enden mit einem vollständig verbundenen Klassifizierer. Den ersten Teil könnte man als die Faltungsbasis (engl. Convolutional Base) des Modells bezeichnen. 
- Bei CNNs werden die neuen Daten der Faltungsbasis eines bereits trainierten NNs übergeben. Anschliessend wird ein neuer Klassifizierer mit deren Ausgabe trainiert.

<img src="Bilder/swapping_fc_classifier.png" width="640" height="440" align="center"/>

### Aber warum nur die Faltungsbasis wiederverwenden? 

Könnte man nicht auch den vollständig verbundenen Klassifizierer wiederverwenden? 

Das sollte im Allgemeinen vermieden werden, weil die von der Faltungsbasis erlernten Repräsentationen
wahrscheinlich allgemeiner und daher besser wiederverwendbar sind: 

- **Die Feature-Maps eines CNNs beschreiben die Vorkommen allgemeiner Konzepte in einem Bild und sind wahrscheinlich unabhängig von der vorliegenden Aufgabe des maschinellen Sehens nützlich.**
- Die vom Klassifizierer erlernten Repräsentationen hingegen beziehen sich notwendigerweise auf die Klassen, mit denen das Modell trainiert wurde. Diese Repräsentationen enthalten lediglich Informationen über die Wahrscheinlichkeit des Vorkommens dieser oder jener Klasse im gesamten Bild. 
- Darüber hinaus enthalten die in den Fully-connected Layern vorhandenen Repräsentationen keine Informationen darüber, wo sich Objekte in den Eingabebildern befinden: Die Layer bewahren keine räumlichen Informationen, die Feature-Maps hingegen enthalten nach wie vor die Positionen der Objekte.

Wenn die Positionen der Objekte für die Lösung einer Aufgabe eine Rolle spielen, sind die vollständig verbundenen Merkmale weitgehend unbrauchbar. Beachten Sie, dass die Verallgemeinerungsfähigkeit (und damit die Wiederverwendbarkeit) der Repräsentationen von der Position eines Layers im Modell abhängt. 
- Die früher auftretenden Layer erzeugen lokale, sehr allgemeine Feature-Maps (die z.B. Ränder, Farben und Texturen enthalten), während die später auftretenden Layer abstraktere Konzepte extrahieren (wie etwa »Katzenohr« oder »Hundeauge«). 
- Sollte sich Ihre neue Datenmenge also sehr von der Datenmenge unterscheiden, mit dem das ursprüngliche Modell trainiert wurde, ist es womöglich besser, wenn nur die ersten paar Layer für die Merkmalsextraktion verantwortlich sind, anstatt die gesamte Faltungsbasis zu nutzen.

Wir verwenden also die Faltungsbasis des mit der ImageNet-Datensammlung trainierten VGG16-Modells, um interessante Merkmale der Hunde- und Katzenbilder zu extrahieren, und trainieren damit anschliessend einen Hunde/Katzen-Klassifizierer.

## Einleitung

In diesem Unterrichtsmaterial zeigen wir, wie Du ein vortrainiertes Convolutional Neural Network (CNN) – konkret die VGG16-Architektur – mit PyTorch Lightning verwenden kannst, um Aufgaben der Bildklassifikation (hier: Hunde vs. Katzen) zu lösen.  
Dabei demonstrieren wir zwei Ansätze:

1. **Schnelle Merkmalsextraktion:**  
   Die Konvolutionsbasis wird einmalig genutzt, um Features für alle Bilder zu berechnen. Diese vorverarbeiteten Features werden dann in einem einfachen, vollständig verbundenen Klassifizierer weiterverwendet.  
   *Vorteil:* Sehr schnelle Trainingszeiten, da die rechenintensive Vorverarbeitung nur einmal erfolgt.  
   *Nachteil:* Datenaugmentation ist in diesem Ansatz nicht möglich.

2. **Merkmalsextraktion mit Datenaugmentation und Feinabstimmung:**  
   Hier erweitern wir die vortrainierte Konvolutionsbasis um einen zusätzlichen Klassifizierer, der direkt an die Eingabedaten angeschlossen wird. So können wir Datenaugmentation im Training nutzen.  
   Nach einer ersten Trainingsphase (mit eingefrorener Konvolutionsbasis) werden einige der oberen Layer "aufgetaut" und gemeinsam mit dem Klassifizierer feinabgestimmt.  
   *Vorteil:* Datenaugmentation reduziert Überanpassung; durch Feinabstimmung kann das Modell weiter verbessert werden.

Im Folgenden findest Du den vollständigen Code, der beide Ansätze mit PyTorch Lightning umsetzt.

## Vorbereitung – Imports und Hilfsfunktionen

Zuerst importieren wir die notwendigen Module und definieren eine Hilfsfunktion zum Glätten von Kurven (nützlich für die Darstellung von Trainings- und Validierungsmetriken).

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets

import lightning as L
from torch.utils.data import DataLoader, TensorDataset


from lightning.pytorch.callbacks import Callback


class MetricsLogger(Callback):
    def __init__(self):
        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []

    def on_train_epoch_end(self, trainer, pl_module):
        self.train_losses.append(trainer.callback_metrics["train_loss"].item())
        self.train_accs.append(trainer.callback_metrics["train_acc"].item())

    def on_validation_epoch_end(self, trainer, pl_module):
        self.val_losses.append(trainer.callback_metrics["val_loss"].item())
        self.val_accs.append(trainer.callback_metrics["val_acc"].item())


def plot_metrics(logger):
    epochs = range(1, min(len(logger.train_losses), len(logger.val_losses)) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, logger.train_losses[: len(epochs)], "bo", label="Training loss")
    plt.plot(epochs, logger.val_losses[: len(epochs)], "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, logger.train_accs[: len(epochs)], "bo", label="Training accuracy")
    plt.plot(epochs, logger.val_accs[: len(epochs)], "b", label="Validation accuracy")
    plt.title("Training and validation accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.show()


## LightningDataModule für den Cats vs. Dogs-Datensatz

Wir gehen davon aus, dass Dein Datensatz in einem Ordner vorliegt, der die Unterordner `train`, `validation` und `test` enthält.

- Für den Trainingsdatensatz definieren wir zusätzlich zur Reskalierung auch Datenaugmentation (Rotation, zufällige Verschiebung, horizontales Spiegeln etc.).
- Für Validierung und Test erfolgt nur die Reskalierung.

Hinweis: Bei vortrainierten Modellen (wie VGG16) ist es oft sinnvoll, auch die ImageNet-Normalisierung zu verwenden – hier nutzen wir jedoch eine vereinfachte Variante (Skalierung auf [0,1]).

In [ ]:
class CatsDogsDataModule(L.LightningDataModule):
    def __init__(self, data_dir, batch_size=20, img_size=150):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size

    def setup(self, stage=None):
        # Transformationen für Training (inklusive Datenaugmentation)
        self.train_transforms = transforms.Compose(
            [
                transforms.Resize((self.img_size, self.img_size)),
                transforms.RandomRotation(40),
                transforms.RandomResizedCrop(self.img_size, scale=(0.8, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),  # skaliert automatisch auf [0,1]
            ]
        )
        # Für Validierung und Test: Nur Resizing und ToTensor
        self.test_transforms = transforms.Compose(
            [
                transforms.Resize((self.img_size, self.img_size)),
                transforms.ToTensor(),
            ]
        )
        self.train_dataset = datasets.ImageFolder(
            os.path.join(self.data_dir, "train"), transform=self.train_transforms
        )
        self.val_dataset = datasets.ImageFolder(
            os.path.join(self.data_dir, "validation"), transform=self.test_transforms
        )
        self.test_dataset = datasets.ImageFolder(
            os.path.join(self.data_dir, "test"), transform=self.test_transforms
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4
        )


# Beispielpfad zum Datensatz (anpassen!)
base_dir = "cats_dogs"
data_module = CatsDogsDataModule(data_dir=base_dir, batch_size=20, img_size=150)


## Teil (a): Schnelle Merkmalsextraktion mittels Pre-Computing der Features

Hier berechnen wir einmalig die Features aller Bilder mit der vortrainierten VGG16-Konvolutionsbasis (ohne Klassifizierer) und speichern diese in Numpy-Arrays. Anschließend trainieren wir einen einfachen vollständig verbundenen Klassifizierer auf diesen vorverarbeiteten Features.

Die VGG16-Konvolutionsbasis wird aus torchvision geladen.  
Die Ausgabe der Konvolutionsbasis hat bei Eingaben von 150×150 in der Regel die Form (Batch, 512, 4, 4), was zu einem Featurevektor der Länge 8192 führt.

In [ ]:
def extract_features(model, dataloader, sample_count):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            inputs, labs = batch
            inputs = inputs.to(next(model.parameters()).device)
            outputs = model(inputs)
            features.append(outputs.cpu())
            labels.append(labs)
            if len(torch.cat(labels)) >= sample_count:
                break
    features = torch.cat(features, dim=0)[:sample_count]
    labels = torch.cat(labels, dim=0)[:sample_count]
    return features, labels


Hier wird ein vortrainiertes, definiertes VGG16-Modell geladen.

In [ ]:
# Laden des vortrainierten VGG16-Modells und Extraktion der Konvolutionsbasis
vgg16_model = torchvision.models.vgg16(pretrained=True)
conv_base = vgg16_model.features  # nur die Faltungsbasis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv_base.to(device)
conv_base.eval()


In [ ]:
# Erstellen eines DataLoaders ohne Datenaugmentation (nur Resizing und ToTensor)
test_dataset_for_features = datasets.ImageFolder(
    os.path.join(base_dir, "train"),
    transform=transforms.Compose(
        [
            transforms.Resize((150, 150)),
            transforms.ToTensor(),
        ]
    ),
)
train_loader_for_features = DataLoader(
    test_dataset_for_features, batch_size=20, shuffle=False, num_workers=4
)

# Festlegen der Sample-Anzahlen (anpassen, falls erforderlich)
train_sample_count = 2000
val_sample_count = 1000
test_sample_count = 1000

train_features, train_labels = extract_features(
    conv_base, train_loader_for_features, train_sample_count
)

# Für Validierung und Test ähnlich:
val_dataset_for_features = datasets.ImageFolder(
    os.path.join(base_dir, "validation"),
    transform=transforms.Compose(
        [
            transforms.Resize((150, 150)),
            transforms.ToTensor(),
        ]
    ),
)
val_loader_for_features = DataLoader(
    val_dataset_for_features, batch_size=20, shuffle=False, num_workers=4
)
val_features, val_labels = extract_features(
    conv_base, val_loader_for_features, val_sample_count
)

test_dataset_for_features = datasets.ImageFolder(
    os.path.join(base_dir, "test"),
    transform=transforms.Compose(
        [
            transforms.Resize((150, 150)),
            transforms.ToTensor(),
        ]
    ),
)
test_loader_for_features = DataLoader(
    test_dataset_for_features, batch_size=20, shuffle=False, num_workers=4
)
test_features, test_labels = extract_features(
    conv_base, test_loader_for_features, test_sample_count
)

# Reshape: von (samples, 512, 4, 4) zu (samples, 8192)
train_features = train_features.view(train_features.size(0), -1)
val_features = val_features.view(val_features.size(0), -1)
test_features = test_features.view(test_features.size(0), -1)


### LightningModule für den Klassifizierer (Feature Extraction)

Der Klassifizierer besteht aus:
- Einer Dense-Schicht mit 256 Neuronen und ReLU-Aktivierung
- Dropout (50%) zur Regularisierung
- Einer finalen Dense-Schicht mit 1 Neuron (binäre Klassifikation)

Als Verlustfunktion verwenden wir Binary Cross Entropy.

In [ ]:
class FeatureExtractionClassifier(L.LightningModule):
    def __init__(self, input_dim=8192):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, 1)
        )
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=2e-5)
        return optimizer


### Training des Klassifizierers auf den vorverarbeiteten Features

Wir erzeugen TensorDatasets für Training, Validierung und Test und trainieren den Klassifizierer über 30 Epochen.

In [ ]:
train_dataset_feat = TensorDataset(train_features, train_labels)
val_dataset_feat = TensorDataset(val_features, val_labels)
test_dataset_feat = TensorDataset(test_features, test_labels)

train_loader_feat = DataLoader(train_dataset_feat, batch_size=20, shuffle=True)
val_loader_feat = DataLoader(val_dataset_feat, batch_size=20, shuffle=False)
test_loader_feat = DataLoader(test_dataset_feat, batch_size=20, shuffle=False)

clf = FeatureExtractionClassifier(input_dim=8192)
logger = MetricsLogger()
trainer = L.Trainer(max_epochs=30, accelerator="auto", devices=1, callbacks=[logger])
trainer.fit(clf, train_loader_feat, val_loader_feat)
trainer.test(clf, test_loader_feat)

plot_metrics(logger)


## Teil (b): Merkmalsextraktion mit Datenaugmentation und Feinabstimmung

Hier erweitern wir das Modell um einen zusätzlichen Klassifizierer, der direkt an die (eingefrorene) Konvolutionsbasis von VGG16 angeschlossen wird.  
Zuerst trainieren wir das Modell mit eingefrorener Konvolutionsbasis. Anschließend "tauten" wir die oberen Layer auf und führen eine Feinabstimmung durch.

### Modellaufbau

Das Modell besteht aus:
- Der VGG16-Konvolutionsbasis (vorgegeben durch `vgg16(pretrained=True).features`)
- Einer Flatten-Schicht
- Einer Dense-Schicht mit 256 Neuronen, ReLU-Aktivierung und Dropout (50%)
- Einer finalen Dense-Schicht mit 1 Neuron (binäre Klassifikation)

Wichtig: Zuerst wird die Konvolutionsbasis eingefroren, damit der neu initialisierte Klassifizierer stabil trainiert. Später werden einige Layer der Konvolutionsbasis wieder aktiviert.

In [ ]:
class FineTuningModule(L.LightningModule):
    def __init__(self):
        super().__init__()
        # Vortrainierte VGG16-Konvolutionsbasis laden
        vgg16_model = torchvision.models.vgg16(pretrained=True)
        self.conv_base = vgg16_model.features
        # Alle Parameter der Konvolutionsbasis einfrieren
        for param in self.conv_base.parameters():
            param.requires_grad = False

        # Klassifizierer definieren
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
        )
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.conv_base(x)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=2e-5)
        return optimizer


### Trainingsphase 1: Klassifizierer trainieren mit eingefrorener Konvolutionsbasis

Wir nutzen hier das DataModule mit Datenaugmentation für den Trainingsdatensatz.

In [ ]:
data_module.setup()
ft_module = FineTuningModule()
logger_ft = MetricsLogger()
trainer_ft = L.Trainer(
    max_epochs=10, accelerator="auto", devices=1, callbacks=[logger_ft]
)
trainer_ft.fit(ft_module, data_module.train_dataloader(), data_module.val_dataloader())

plot_metrics(logger_ft)


#### Data Augmentation Beispiele

In [ ]:
import matplotlib.pyplot as plt


def show_augmented_images(dataset, num_images=6):
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        image, label = dataset[i]
        axes[i].imshow(image.permute(1, 2, 0))
        axes[i].axis("off")
    plt.show()


show_augmented_images(data_module.train_dataset)


### Trainingsphase 2: Feinabstimmung – Auftauen der obersten Schichten

Jetzt taue die oberen Convolutional Layer der Konvolutionsbasis (z.B. ab Index 24, was in VGG16 typischerweise Block 5 entspricht) auf, damit diese gemeinsam mit dem Klassifizierer trainiert werden können.  
Wir definieren dazu ein neues LightningModule, in dem die Layer ab Index 24 wieder trainierbar gemacht werden und die Lernrate auf 1e-5 reduziert wird.

In [ ]:
class FineTuningModule_Unfrozen(L.LightningModule):
    def __init__(self):
        super().__init__()
        vgg16_model = torchvision.models.vgg16(pretrained=True)
        self.conv_base = vgg16_model.features
        # Zunächst alle Parameter einfrieren
        for param in self.conv_base.parameters():
            param.requires_grad = False
        # Ab Index 24 (Block 5) wieder freigeben
        for layer in self.conv_base[24:]:
            for param in layer.parameters():
                param.requires_grad = True

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
        )
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.conv_base(x)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().unsqueeze(1)
        logits = self(x)
        loss = self.loss_fn(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = (preds.float() == y).float().mean()
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=1e-5)
        return optimizer


Trainiere nun das Modell mit den aufgetauten Schichten (Feinabstimmung) über z.B. 100 Epochen.

In [ ]:
ft_module_unfrozen = FineTuningModule_Unfrozen()
logger_ft_fine = MetricsLogger()
trainer_ft_fine = L.Trainer(
    max_epochs=30, accelerator="auto", devices=1, callbacks=[logger_ft_fine]
)
trainer_ft_fine.fit(
    ft_module_unfrozen, data_module.train_dataloader(), data_module.val_dataloader()
)
plot_metrics(logger_ft_fine)
# Optional: Modell abspeichern
torch.save(ft_module_unfrozen.state_dict(), "cats_and_dogs_small_finetuned.pt")


### Evaluation auf den Testdaten

Nach Abschluss des Trainings evaluieren wir das finale Modell auf den Testdaten.

In [ ]:
trainer_ft_fine.test(ft_module_unfrozen, data_module.test_dataloader())


In [ ]:
def show_predictions(model, dataloader, num_images=6):
    model.eval()
    images, labels = [], []
    for batch in dataloader:
        batch_images, batch_labels = batch
        for img, lbl in zip(batch_images, batch_labels):
            if len(images) < num_images // 2 and lbl == 0:
                images.append(img)
                labels.append(lbl)
            elif len(images) < num_images and lbl == 1:
                images.append(img)
                labels.append(lbl)
            if len(images) == num_images:
                break
        if len(images) == num_images:
            break

    images = torch.stack(images)
    labels = torch.tensor(labels)
    with torch.no_grad():
        outputs = model(images.to(next(model.parameters()).device))
        preds = torch.sigmoid(outputs).cpu() > 0.5

    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        axes[i].imshow(images[i].permute(1, 2, 0))
        axes[i].set_title(f"Pred: {preds[i].item()}, Label: {labels[i].item()}")
        axes[i].axis("off")
    plt.show()


show_predictions(ft_module_unfrozen, data_module.test_dataloader())


## Zusammenfassung

- **CNNs** sind hervorragend für Aufgaben des maschinellen Sehens geeignet.
- Bei kleinen Datenmengen kann Überanpassung ein Problem sein – Datenaugmentation ist hier eine effektive Lösung.
- Mithilfe der **Merkmalsextraktion** können vortrainierte Modelle (wie VGG16) effizient wiederverwendet werden, indem nur der Klassifizierer neu trainiert wird.
- Die anschließende **Feinabstimmung** erlaubt es, die spezialisierten Merkmale des vortrainierten Modells an die neue Aufgabe anzupassen und so die Leistung weiter zu verbessern.

Dieses Material zeigt, wie Du mit PyTorch Lightning einen strukturierten Deep-Learning-Workflow (von der Datenvorbereitung über die schnelle Merkmalsextraktion bis hin zur Feinabstimmung) realisieren kannst.

Viel Erfolg beim Experimentieren und Lernen!